In [1]:
import os
import sys

def add_to_sys_path(relative_path):
    abs_path = os.path.abspath(os.path.join(os.getcwd(), relative_path))
    if abs_path not in sys.path:
        sys.path.append(abs_path)

add_to_sys_path(os.path.join('..', 'Utils'))
add_to_sys_path(os.path.join('..', 'Model'))
add_to_sys_path(os.path.join('..', 'alg'))

In [2]:
import pandas as pd
import numpy as np
import random
import shap

import matplotlib.pyplot as plt
import plotly.graph_objects as go
import seaborn as sns

from algorithm import *
from explainer import *
from genetic import *
import dalex as dx
from LR import base_model as base_model_call
from model_data_processing import Data_Handler, Model_Metrics_Visualizer

/home/jn/anaconda3/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
2025-06-25 20:57:15.775603: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-06-25 20:57:16.855475: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
# Assuming your Explainer and Algorithm classes are defined as above
# And you have x_train, protected_feature, I (your row_id integer)

# --- Example Setup (replace with your actual data) ---
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import shap
import warnings # Ensure warnings is imported for the conditional logic

# Dummy data
X = pd.DataFrame(np.random.rand(100, 5), columns=[f'f{i}' for i in range(5)])
y = pd.Series(np.random.randint(0, 2, 100))
protected_feature = 'f0'
I = 5 # Example row_id

# Train a dummy model (e.g., Logistic Regression)
model = LogisticRegression(random_state=42)
model.fit(X, y)

# Create explainer instance
explainer = Explainer(model, X) # X is your background data

# --- Your problematic code section, adjusted ---

# FIX: Remove 'alpha' and 'pop_count' if Algorithm.__init__ doesn't accept them
a = Algorithm(explainer, constant=[protected_feature], row_id=I)

# Get SHAP values for the *specific row* 'a' is focused on (self._x)
# a.fool() will populate a.result_explanation['original'] with the SHAP values for self._x
a.fool()
t_original_shap = a.result_explanation['original'] # This is now a 1D array (num_features,)

# Now, 't_original_shap' is a 1D array, so the list comprehension will work element-wise
# t_original_shap[0] and len(t_original_shap) will work correctly.
# The `t[0] / (len(t) - 3)` term implies some special handling for the first feature,
# and assumes at least 4 features total for `len(t) - 3` to not be <= 0.
# Make sure len(t) - 3 > 0, i.e., len(t) > 3 features.
if len(t_original_shap) <= 3:
    warnings.warn("Number of features is too small for target calculation (len(t) - 3). Adjusting logic.")
    divisor = 1 # Avoid division by zero or negative
else:
    divisor = (len(t_original_shap) - 3)


t_target = np.array([
    val + (t_original_shap[0] / divisor) if val > 0 else val - (t_original_shap[0] / divisor)
    for val in t_original_shap
])

# Assuming you want to set the first feature's target specifically
t_target[0] = -0.005

# Now create the DataFrame
pd.DataFrame({
    'variable': a.explainer.data.columns,
    'shap_values': t_original_shap,
    'target': t_target
})

,variable,shap_values,target
0,f0,-0.087694,-0.005000
1,f1,-0.243119,-0.199272
2,f2,0.043507,-0.000340
3,f3,0.249593,0.205746
4,f4,-0.025892,0.017955
